## Python Logging - Deep Dive
<b>24th  November, 2020</b>

---
### Content
* Why logging?
* 5 levels of logging
* What to include?
* Basic example
  * How to log to a file instead of console?
  * How to change logging format?
* What is and how to setup FileHandler and Formatter?
* How to include traceback information in logged messages?
* Using Filters
* Using config files
* Best practices and points to remember


---
### Why Logging?
Logging is a more structured and robust approach to finding what part of the script was executed, at what time and in certain cases inspect what values the variables hold. It's advantage during prototyping as well as in production is immense as you can have a log with all previous errors from your code as reference which will help you to replicate them for analysis.

Using logging, you can:
* Control message level to log only required ones
* Control where to show or save the logs
* Control how to format the logs with built-in message templates
* Know which module/line the message is coming from

---
### 5 Levels of logging
Every message you log is called a LogRecord. By default there are 5 levels of logs. You should classify your LogRecord to one of these levels depending on the severity: 
* DEBUG: Detailed information, for diagnosing problems.
* INFO: Confirm things are working as expected.
* WARNING: Something unexpected happened, or indicative of some problem. But the software is still working as expected.
* ERROR: More serious problem, the software is not able to perform some function.
* CRITICAL: A serious error, the program itself may be unable to continue running.

**Note:**
* By default, the logging module logs the messages with a severity level of WARNING or above (i.e. DEBUG and INFO are not logged)
* You can set your own custom level of logging as well. 

---
### What to include?
This is completely upto you depending on your use case. However, your log should typically include atleast the following information:
* When - timestamp of the logged event
* Who - the application/logger name 
* Where - line number or the function/module name
* What - log level and the message

**Note:** 
* Keep in mind not to include any sensitive information in your logs

---
### Basic Example

Few lines have been commented out. You can uncomment those and run to see how your output changes.

In [51]:
import logging

# This is optional and sets the basic configuration for the root logger (Sets everything to default values)
logging.root.handlers = []
logging.basicConfig(
    # filename = 'sample.log',  # path to the log file
    # format = '%(asctime)s - %(name)s - %(lineno)d - %(levelname)s - %(message)s',  # logging format 
    # datefmt = '%d-%b %H:%M:%S',  # format for date-time
    level = logging.WARNING,  # sets the minimum level of log severity
)

In [59]:
logger = logging.getLogger()  # setup a new logger object (By default if no name is given it gets the root logger)
logger.setLevel(logging.INFO)  # sets the minimum level of log severity that the logger accepts (if root logger, this line overwrites any setting given in basicConfig)

# function to calculate hypotenuse
def hypotenuse(a, b):

  # logs a warning message if input is non positive
  if a<=0: logger.warning(f'a is non positive := {a}')
  if b<=0: logger.warning(f'a is non positive := {b}')

  return (a**2 + b**2)**0.5

In [60]:
a = 0
b = 4
logger.info(f'Hypotenuse of {a}, {b} is {hypotenuse(a, b)}')

INFO:root:Hypotenuse of 0, 4 is 4.0


In [61]:
# Importing module1.py
import module1

*module1.py* contains only the following lines of code
```
import logging

logger = logging.getLogger(__name__)

def trail():
  logger.info(f'Hello from the other side!')
```

Notice the variable *\_\_name__* in *logging.getLogger()*. This automatically gets sets the logger name to filename (in this case module1)


In [62]:
module1.trail()  #  verify that logger name has changed to module1

INFO:module1:Hello from the other side!


* Internally, the message is turned into a LogRecord object and routed to a Handler object registered for this logger. The handler will then use a Formatter to turn this LogRecord into a string and emit that string. 
* In general, a module should only emit log messages and should not be configured how those messages are handled. That should be the responsibility of the application.
  * Notice that in module1 only the logger object is created and no formatting or configuration is set. It picks up these configurations from the current file.
* **setLevel()** method for a logger determines which severity of messages it will pass to its handlers

---
### FileHandler and Formatter

The task of a handler is to take the LogRecord from the logger, apply formatting and then pass that string to its final destination (a log file,screen etc).

The FileHandler() and Formatter() classes are used to setup the output file and format of the messages for loggers.
* *logging.basicConfig()* was used to setup the filename and format of the messages in the root logger.
* Custom loggers will fallback and write to the file set by the root logger unless you set a handler for your custom logger.
* There is a **setLevel()**  method for handlers as well
  * A logger can have multiple handlers. You can set a certain handler to log all messages while another handler to log only ERROR messages

There are other types of handlers as well which can be used for various purposes. Some common ones:
* StreamHandler: instead of logging to a file, it displays the log on the console
* SMTPHandler: send an email (mostly used for sending critical logs)
* TimeRotatingFileHandler: creates a new log file after fixed time interval (eg. weekly, daily)

Full list of handlers: https://docs.python.org/3.9/library/logging.handlers.html

In [66]:
logger2 = logging.getLogger('log2')  # setting a new logger
logger2.info('Hi')  # logs messages in the default format (same as root logger)

INFO:log2:Hi


In [67]:
# handler = logging.FileHandler('file.log')
handler = logging.StreamHandler()  # creates a handler object
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(funcName)s - %(message)s')  # creates a formatter object
handler.setFormatter(formatter)  # sets the formatter for the handler
logger2.handlers = []  # removes all custom handlers 
logger2.addHandler(handler)  # adds the handler to the logger
logger2.info('Hi')  # As a new handler is added, hence multiple logging

2020-11-25 11:16:09,766 - INFO - <module> - Hi
INFO:log2:Hi


In [68]:
handler.setLevel(logging.WARNING)  # logging level of the handler can be changed on the fly
logger2.info('Hi')  # This message is not logged by the new handler

INFO:log2:Hi


---
### Traceback messages
* You can include the entire error message with traceback using *exc_info=True* parameter
* *logging.exception()* logs a message with level ERROR and adds exception information to the message. It is like calling *logging.error(exc_info=True)*

In [69]:
def divide(a, b):
  logger2.info(f'Dividing {a} by {b}')
  try:
    return a/b
  except:
    # logger2.error(f'Division by 0', exc_info=True)
    logger2.exception(f'Division by 0')  # works exactly like the above line

In [70]:
logger2.handlers = []
divide(2,0)

INFO:log2:Dividing 2 by 0
ERROR:log2:Division by 0
Traceback (most recent call last):
  File "<ipython-input-69-19587b5edd7c>", line 4, in divide
    return a/b
ZeroDivisionError: division by zero


---
### Using Filters
Filters provide extra logic to determine which log messages to output. It is a step after checking the basic logging level, but before passing the log message to the handlers. The output of the filter should be either True or False. You can even use filters to make new attributes which you can use in your formatting.


The flow of the logic is something like this:
* The log message is executed.
* Logger checks the severity of the LogRecord. If it is less severe than the level set then it is not logged.
* Logger applies the filter. If it doesn't meet the filter condition, it is not logged.
* Handler checks the severity of the LogRecord. If it is less severe than the level set then it is not logged.

In [71]:
class CustomFilter(logging.Filter):
  # a dictionary to map levels to color codes 
  COLOR = {
      'DEBUG': 'GREEN',
      'INFO': 'GREEN',
      'WARNING': 'YELLOW',
      'ERROR': 'RED',
      'CRITICAL': 'RED',
  }

  def filter(self, record):
    record.color = CustomFilter.COLOR[record.levelname]  # setting a custom attribute 'color' based on severity level
    
    # logic for only accepting lines that are below line number 3
    if record.lineno > 3:
      return False
    else:
      return True

In [79]:
handler = logging.StreamHandler()
formatter = logging.Formatter('%(color)s - %(message)s')
handler.setFormatter(formatter)
logger2.handlers = []
logger2.addHandler(handler)
logger2.addFilter(CustomFilter())  # applying custom filter to the logger

In [80]:
logger2.warning('This should appear')


logger2.warning('This should not appear')

YELLOW - This should appear


---
### Configuration
You can configure logging as shown above using the module and class functions or by creating a config file or a dictionary and loading it using *fileConfig()* or *dictConfig()* respectively. These are useful in case you want to change your logging configuration in a running application.

---
### Best Practices and Points to Remember
* Use different loggers for different sources
* Use different handlers for different destinations/formats
* Use *\_\_name__* variable while naming loggers in modules so that they take the name of the module automatically
* Avoid using custom levels of logging and try sticking to the default 5 levels
* *setLevel()* can be used to change the minimum severity of loggers and handlers on the fly
* *logging.exception()* should only be called using an exception handler as this method always dumps exception information
* *basicConfig()* is optional. However, if not used, it is important to set a handler or else you might not receive INFO and DEBUG messages even after changing severity level of the logger.
* By default, the *basicConfig()* sets does the following:
  * sets a StreamHandler()
  * sets the log format like '%(levelname)s:%(name)s:%(message)s'
  * level is set to logging.WARNING
* Repeated calls to *basicConfig()* are ignored by default. You have to remove the handler configured during the previous *basicConfig()* call in order to excecute it again.
```
logging.root.handlers = [] 
logging.basicConfig()
```